<a href="https://colab.research.google.com/github/amitgupta226571/DEEP-LEARNING-/blob/main/Experiment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from nltk.translate.bleu_score import corpus_bleu
from collections import Counter


In [34]:
path = "/content/drive/MyDrive/Data/spa.txt"

pairs = []

with open(path, encoding="utf-8") as f:
    for line in f:
        if "\t" in line:
            eng, spa = line.strip().split("\t")
            pairs.append((eng.lower(), spa.lower()))

print("Total pairs:", len(pairs))

# Sample if large
pairs = pairs[:10000]

random.shuffle(pairs)

train_size = int(0.8 * len(pairs))
val_size = int(0.1 * len(pairs))

train_pairs = pairs[:train_size]
val_pairs = pairs[train_size:train_size+val_size]
test_pairs = pairs[train_size+val_size:]


Total pairs: 8343


In [35]:
PAD, SOS, EOS, UNK = "<pad>", "<sos>", "<eos>", "<unk>"

def build_vocab(pairs, index):
    counter = Counter()
    for pair in pairs:
        counter.update(pair[index].split())

    vocab = {PAD:0, SOS:1, EOS:2, UNK:3}

    for word in counter:
        vocab[word] = len(vocab)

    return vocab

eng_vocab = build_vocab(train_pairs, 0)
spa_vocab = build_vocab(train_pairs, 1)

print("English vocab:", len(eng_vocab))
print("Spanish vocab:", len(spa_vocab))


English vocab: 2552
Spanish vocab: 4820


In [36]:
def numericalize(vocab, sentence):
    return [vocab.get(word, vocab[UNK]) for word in sentence.split()]

def prepare_data(pairs):
    data = []
    for eng, spa in pairs:
        src = [eng_vocab[SOS]] + numericalize(eng_vocab, eng) + [eng_vocab[EOS]]
        trg = [spa_vocab[SOS]] + numericalize(spa_vocab, spa) + [spa_vocab[EOS]]
        data.append((torch.tensor(src), torch.tensor(trg)))
    return data

train_data = prepare_data(train_pairs)
val_data = prepare_data(val_pairs)
test_data = prepare_data(test_pairs)


In [37]:
def collate_fn(batch):
    src_batch = [item[0] for item in batch]
    trg_batch = [item[1] for item in batch]

    src_batch = pad_sequence(src_batch, padding_value=eng_vocab[PAD])
    trg_batch = pad_sequence(trg_batch, padding_value=spa_vocab[PAD])

    return src_batch, trg_batch

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=32, collate_fn=collate_fn)


In [38]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, hidden, cell


In [39]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(0))
        return prediction, hidden, cell


In [40]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)
        input = trg[0,:]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(len(eng_vocab), 256, 512)
decoder = Decoder(len(spa_vocab), 256, 512)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=spa_vocab[PAD])

def train(model, loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for src, trg in loader:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()

            output = model(src, trg)

            output_dim = output.shape[-1]
            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader)}")


In [42]:
def evaluate_bleu(model, data):
    model.eval()
    references = []
    hypotheses = []

    with torch.no_grad():
        for src, trg in data:
            src = src.unsqueeze(1).to(device)
            trg = trg.unsqueeze(1).to(device)

            output = model(src, trg, 0)
            pred_tokens = output.argmax(2).squeeze().tolist()
            trg_tokens = trg.squeeze().tolist()

            hypotheses.append(pred_tokens)
            references.append([trg_tokens])

    return corpus_bleu(references, hypotheses)


In [43]:
plt.figure(figsize=(8,6))
plt.matshow(attention_weights.cpu().detach().numpy())
plt.colorbar()
plt.show()


NameError: name 'attention_weights' is not defined

<Figure size 800x600 with 0 Axes>